In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import numpy
import spacy
import re
from utils.ngram import ngram

In [7]:
nlp = spacy.load("en_core_web_sm")
yeetus = nlp("Ryan \n. Yes okay ")
for each in yeetus:
    print(each.text)

Ryan


.
Yes
okay


In [22]:
if (1):
    rf = open("./data/hp0.txt", "r", encoding="utf8")
    wf = open("./data/hp1.txt", "w", encoding="utf8")
    for line in rf.readlines():
        if (re.match("Page | . Harry Potter and the Philosophers Stone - J.K. Rowling", line) == None and line != "\n"):
            wf.write(line.replace("\n", " "))
    rf.close()
    wf.close()

In [24]:
def remove_puncs(doc): # doc output is list
    output = []
    for token in doc:
        if (not token.is_punct or token.text != " "):
            output.append(token)
    return output

nlp = spacy.load("en_core_web_sm")
rf = open("./data/hp1.txt", "r", encoding="utf8")
doc = nlp(rf.read())
doc = remove_puncs(doc)

In [25]:
word_to_int = create_word_to_int(doc)
int_to_word = create_int_to_word(word_to_int)
print("word_to_int and int_to_word create done")

word_to_int and int_to_word create done


In [26]:
print(len(doc))
print(word_to_int["THE"])
print(int_to_word[6692])

109461
720
sheep


In [27]:
word_ngram = ngram(doc, 7, word_to_int)
print("create n gram done")

create n gram done


In [28]:
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

In [29]:
import tempfile
print(tempfile.gettempdir())

C:\Users\Alex\AppData\Local\Temp


In [30]:
print(word_ngram[0:10])

[['THE BOY WHO LIVED Mr. and Mrs. ', 4772], ['BOY WHO LIVED Mr. and Mrs. Dursley', 4962], ['WHO LIVED Mr. and Mrs. Dursley, ', 3658], ['LIVED Mr. and Mrs. Dursley, of ', 1737], ['Mr. and Mrs. Dursley, of number ', 4273], ['and Mrs. Dursley, of number four', 4962], ['Mrs. Dursley, of number four, ', 4370], ['Dursley, of number four, Privet ', 4409], [', of number four, Privet Drive', 4962], ['of number four, Privet Drive, ', 1440]]


In [31]:
embedded_ngram = embed(list(zip(*word_ngram))[0])
labels = list(zip(*word_ngram))[1]

In [32]:
print(len(word_ngram))
print(embedded_ngram.shape)

109454
(109454, 512)


In [33]:
inputs = keras.Input(shape=(512,))
num_classes = len(word_to_int)
x = layers.Dense(3000, activation="relu")(inputs)
outputs = layers.Dense(num_classes, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 512)]             0         
_________________________________________________________________
dense (Dense)                (None, 3000)              1539000   
_________________________________________________________________
dense_1 (Dense)              (None, 6693)              20085693  
Total params: 21,624,693
Trainable params: 21,624,693
Non-trainable params: 0
_________________________________________________________________


In [34]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=3e-4),
              loss=keras.losses.SparseCategoricalCrossentropy())


In [35]:
model.fit(embedded_ngram.numpy(), numpy.array(labels),
          batch_size=32, epochs=10)

Epoch 1/10
3421/3421 [==============================] - 344s 100ms/step - loss: 6.2158
Epoch 2/10
3310/3421 [============================>.] - ETA: 11s - loss: 5.4618

KeyboardInterrupt: 

In [8]:
embedded = embed(['One day in July,'])
outputboi = model.predict(embedded.numpy()) # 1 * 6000
print(outputboi.shape)
outputboi = tf.math.argmax(outputboi, axis=1)
print(int_to_word_loaded[int(outputboi.numpy()[0])])

(1, 6675)
The


In [7]:
model = keras.models.load_model('xd/xd')

In [6]:
word_to_int_loaded = pickle.load(open("word2int.p", "rb"))
int_to_word_loaded = create_int_to_word(word_to_int_loaded)